In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from stockMarket.core import get_tickers_from_index
from stockMarket.api import populate_contracts

tickers = get_tickers_from_index('sp500')

In [13]:
from stockMarket.technicals.strategy import Strategy

strategy = Strategy(tickers)
strategy.screen()

0 0 11


  2%|▏         | 11/503 [00:10<07:35,  1.08it/s]


KeyboardInterrupt: 

In [9]:
from stockMarket.technicals.technicals import Technicals

for ticker in tickers:
    n_bars = 200
    contract = Technicals(ticker=ticker)
    contract.init_pricing_data(interval="weekly", n_bars=n_bars)
    for time in range(20, 52):
        time = -time
        
        if len(contract.pricing_data) < abs(time):
            continue
        
        last_candle = contract.pricing_data.iloc[time]

        candle_range = last_candle.high - last_candle.low
        fib_30 = last_candle.low + candle_range * 0.3
        fib_70 = last_candle.high - candle_range * 0.3
        
        ema_8 = contract.calc_ema(8)
        ema_20 = contract.calc_ema(20)
        ema_50 = contract.calc_ema(50)
        
        candle_classification = None
        if last_candle.open < fib_30 and last_candle.close < fib_30:
            candle_classification = "bearish"
        elif last_candle.open > fib_70 and last_candle.close > fib_70:
            candle_classification = "bullish"
            
        if candle_classification:
            print(f"{ticker} is showing a {candle_classification} candle on {last_candle.name}")
            
        ema_trend = False
        if candle_classification == "bearisch":
            ema_trend = ema_8[time] < ema_20[time] < ema_50[time]
            ema_trend &= last_candle.high >= ema_8[time]
        elif candle_classification == "bullish":
            ema_trend = ema_8[time] > ema_20[time] > ema_50[time]
            ema_trend &= last_candle.low <= ema_8[time]
        
        if ema_trend:
            print(f"    {ticker} {candle_classification} buy signal")

MHK is showing a bullish candle on 2023-07-03 15:30:00
LW is showing a bearish candle on 2023-07-31 15:30:00
LW is showing a bullish candle on 2023-07-10 15:30:00
    LW bullish buy signal
LW is showing a bullish candle on 2023-06-26 15:30:00
    LW bullish buy signal
LW is showing a bullish candle on 2023-05-15 15:30:00
META is showing a bearish candle on 2023-10-09 15:30:00
META is showing a bearish candle on 2023-09-11 15:30:00
META is showing a bullish candle on 2023-06-26 15:30:00
DGX is showing a bearish candle on 2023-09-11 15:30:00
STT is showing a bearish candle on 2023-10-16 15:30:00
STT is showing a bullish candle on 2023-08-07 15:30:00
INTU is showing a bearish candle on 2023-10-09 15:30:00
INTU is showing a bullish candle on 2023-05-30 15:30:00
CCI is showing a bullish candle on 2023-10-02 15:30:00
CCI is showing a bearish candle on 2023-08-28 15:30:00
CCI is showing a bearish candle on 2023-07-24 15:30:00
CCI is showing a bullish candle on 2023-03-20 14:30:00
MMC is showi

KeyboardInterrupt: 